In [261]:
!python -m pip install --upgrade pip

In [262]:
import os
import time
import codecs

import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager


In [263]:
USERNAME = "amelia99"
PASSWORD = os.environ.get('password') 
if PASSWORD is None:
    PASSWORD = input("Please enter your password: ")

In [264]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
MAIN_URL = "https://farmrpg.com/"
driver.get(MAIN_URL)

In [287]:
location_urls = pd.read_csv("location_urls.csv")
location_urls

,name,url
0,home,https://farmrpg.com/
1,farm,https://farmrpg.com/#!/xfarm.php?id=343807
2,explore,https://farmrpg.com/#!/explore.php
3,mount banon,https://farmrpg.com/#!/area.php?id=8
4,forest,https://farmrpg.com/#!/area.php?id=7
5,help,https://farmrpg.com/#!/quests.php
6,inventory,https://farmrpg.com/#!/inventory.php
7,login,https://farmrpg.com/#!/login.php
8,home,https://farmrpg.com/#!/index.php
9,fish,https://farmrpg.com/#!/fish.php


In [288]:
location_xpaths = pd.read_csv("location_xpaths.csv", delimiter=";")
location_xpaths

,name,xpath
0,farm,"//div[@class=""view view-main navbar-through""]/..."
1,inventory,"//div[@class=""view view-main navbar-through""]/..."
2,explore,"//div[@class=""view view-main navbar-through""]/..."
3,fish,"//div[@class=""view view-main navbar-through""]/..."


In [267]:
def get_current_location():
    print(driver.current_url)

    try:
        return location_urls[location_urls["url"] == driver.current_url].iloc[0,0]
    except:
        print(f"Location {driver.current_url} not found in locations.csv")
        return None

get_current_location()

https://farmrpg.com/


'home'

In [268]:
def is_refresh_button_available():
    return bool(len(driver.find_elements(By.XPATH, "//i[@aria-label='Refresh Game']")))

In [270]:
def goto(target_location):
    target_location = target_location.lower()
    current_location = get_current_location()
    if current_location != target_location:
        print(f"GOTO:  {current_location} -> {target_location}")
        
        driver.get(MAIN_URL)
        
        if target_location == "home":
            return
        elif target_location in location_xpaths["name"].values.tolist():
            xpath =  location_xpaths.loc[location_xpaths["name"] == target_location,"xpath"].iloc[0]
            print(xpath)
            WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath)))
            driver.find_element(By.XPATH, xpath).click()
        else:
            print("Sorry, this Location has not been added yet!")

# goto("inventory")
# location_xpaths["name"].values.tolist()

In [271]:
def is_logged_in():
    goto("home")

    if driver.find_element(By.XPATH, "//div[@id='logged_in_username']").get_attribute("innerHTML") == USERNAME:
        return True
    return False

In [272]:
def login_user(username, password):
    if is_logged_in():
        print(f"Already logged in as {username}")
        return

    login_div = driver.find_element(By.XPATH, "//div[contains(text(), 'Login')]")
    login_div.click()

    time.sleep(1)

    username_input = driver.find_element(By.XPATH, "//input[@name='username']")
    username_input.send_keys(username)

    password_input = driver.find_element(By.XPATH, "//input[@name='password']")
    password_input.send_keys(password)

    login_button = driver.find_element(By.XPATH, "//input[@value='Login']")
    login_button.click()

    time.sleep(1)

    if is_logged_in():
        print(f"Successfully logged in as {username}")
    else:
        login_user(username, password)

login_user(USERNAME, PASSWORD)


https://farmrpg.com/
https://farmrpg.com/index.php
Location https://farmrpg.com/index.php not found in locations.csv
GOTO:  None -> home
Successfully logged in as amelia99


In [273]:
def set_farm_url():
    goto("home")
    
    farm_id = driver.find_element(By.XPATH, "//a[contains(@href, 'xfarm.php?id')]").get_attribute("href").split("id=")[-1]
    locations.loc[locations["name"] == "farm","url"] = f"https://farmrpg.com/#!/xfarm.php?id={farm_id}"
    print(f"Set Farm URL to {locations.loc[locations['name'] == 'farm','url'].iloc[0]}")

set_farm_url()
locations
    

https://farmrpg.com/
Set Farm URL to https://farmrpg.com/#!/xfarm.php?id=343807


,name,url
0,home,https://farmrpg.com/
1,farm,https://farmrpg.com/#!/xfarm.php?id=343807
2,explore,https://farmrpg.com/#!/explore.php
3,mount banon,https://farmrpg.com/#!/area.php?id=8
4,forest,https://farmrpg.com/#!/area.php?id=7
5,help,https://farmrpg.com/#!/quests.php
6,inventory,https://farmrpg.com/#!/inventory.php
7,login,https://farmrpg.com/#!/login.php
8,home,https://farmrpg.com/#!/index.php


In [275]:
def harvest():
    goto("farm")
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Harvest All')]")))
    time.sleep(0.01)
    try:
        driver.find_element(By.XPATH, '//a[contains(text(), "Harvest All")]').click()
    except StaleElementReferenceException:
        driver.refresh()
        driver.find_element(By.XPATH, '//a[contains(text(), "Harvest All")]').click()

harvest()

https://farmrpg.com/#!/xfarm.php?id=343807


In [276]:
def get_available_crops(crop_select=None):
    goto("farm")
    
    if crop_select is None:
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//select[@class='seedid inlineinputlg']")))
        crop_select = Select(driver.find_element(By.XPATH, '//select[@class="seedid inlineinputlg"]'))

    return pd.DataFrame([[o.get_attribute("data-name"), o.get_attribute("data-amt")] for o in crop_select.options], columns=["name", "amount"]).dropna()

get_available_crops()

https://farmrpg.com/#!/xfarm.php?id=343807


,name,amount
1,Cabbage Seeds,43
2,Carrot Seeds,508
3,Cucumber Seeds,341
4,Eggplant Seeds,70
5,Gold Carrot Seeds,5
6,Hops Seeds,150
7,Leek Seeds,341
8,Pea Seeds,174
9,Pepper Seeds,114
10,Pumpkin Seeds,3


In [277]:
def plant_crop(crop_name):
    goto("farm")
    
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//select[@class='seedid inlineinputlg']")))
    crop_select = Select(driver.find_element(By.XPATH, '//select[@class="seedid inlineinputlg"]'))

    available_crops = get_available_crops(crop_select=crop_select)

    crop_name = crop_name[0].upper() + crop_name[1:].lower()
    if "seeds" in crop_name.lower():
        crop_name = crop_name.replace("seeds","").replace("Seeds","").replace(" ","")
    crop_name += " Seeds"


    crop_info = available_crops[available_crops.name == crop_name]

    if crop_info.empty:
        print(f"Could not find crop {crop_name} in available crops")
        # TODO: implement buying
        return

    crop_select.select_by_index(crop_info.index.values[0])

    driver.find_element(By.XPATH, '//a[contains(text(), "Plant All")]').click()
    print(f'Planted {crop_name}')
    
plant_crop("carrot")

https://farmrpg.com/#!/xfarm.php?id=343807
https://farmrpg.com/#!/xfarm.php?id=343807
Planted Carrot Seeds


In [278]:
def is_harvest_available():
    goto("home")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//a[contains(@href, 'xfarm.php?id')]//span[contains(@class,'ready ready')]")))
    message = driver.find_element(By.XPATH, "//a[contains(@href, 'xfarm.php?id')]//span[contains(@class,'ready ready')]").get_attribute("innerHTML")

    if message == "":
        print("No Crops planted!")
        return False
    elif " Growing" in message:
        print("Crops are still growing!")
        return False
    elif " READY!" in message:
        print(message.replace('<span style="color:#30d611">',"").replace("</span>",""))
        return True

is_harvest_available()
    

https://farmrpg.com/#!/xfarm.php?id=343807
GOTO:  farm -> home
Crops are still growing!


False

In [291]:
def get_explore_locations():
    goto("explore")

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")
    return [location.get_attribute("innerHTML").split("<br>")[0] for location in locations]

get_explore_locations()

https://farmrpg.com/#!/fish.php
GOTO:  fish -> explore
//div[@class="view view-main navbar-through"]//a[@href='explore.php']


['Forest',
 'Small Cave',
 'Small Spring',
 'Haunted House',
 'Highland Hills',
 'Cane Pole Ridge',
 'Misty Forest',
 'Black Rock Canyon',
 'Mount Banon']

In [282]:
def explore(target_location=None, num=5, rate=10):
    goto("explore")
        
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'area')]//div[@class='item-title']")
    
    if target_location is None:
        locations[-1].click()

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@id='exploreconsole']")))
    tab = driver.find_element(By.XPATH, "//div[@id='exploreconsole']")

    for i in range(num):
        tab.click()
        time.sleep(1/rate)

explore_highest(num=100)

https://farmrpg.com/#!/explore.php


In [294]:
def get_fish_locations(target_location=None):
    goto("fish")
        
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'fishing.php')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'fishing.php')]//div[@class='item-title']")
    return [location.get_attribute("innerHTML").split("<br>")[0] for location in locations]

get_fish_locations()

https://farmrpg.com/#!/fishing.php?id=8
GOTO:  vast ocean -> fish
//div[@class="view view-main navbar-through"]//a[@href='fish.php']


['Farm Pond',
 'Small Pond',
 'Forest Pond',
 'Lake Tempest',
 'Small Island',
 'Crystal River',
 'Emerald Beach',
 'Vast Ocean']

In [319]:
def fish_manually(target_location=None, num=5):
    goto("fish")
        
    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//ul//a[contains(@href,'fishing.php')]//div[@class='item-title']")))
    locations = driver.find_elements(By.XPATH, "//ul//a[contains(@href,'fishing.php')]//div[@class='item-title']")
    
    if target_location is None:
        locations[-1].click()

    WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@id='fishinwater']")))

    fish_divs = driver.find_elements(By.XPATH, "//img[@src='/img/items/fish.png']")
    print(fish_divs)

    def fish_coming_up(driver):
        for fish in fish_divs:
            if "catch" in fish.get_attribute("class"):
                print(fish.get_attribute("outerHTML"))
                return fish
    
    time.sleep(1)
    for i in range(num):
        catchable = WebDriverWait(driver, 1).until(fish_coming_up)
        ActionChains(driver).move_to_element(catchable.find_element(By.XPATH, '..')).click().perform()

        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, "//div[@class='fishcaught finalcatch1b']")))
        ActionChains(driver).move_to_element(driver.find_element(By.XPATH, "//div[@class='fishcaught finalcatch1b']")).click().perform()
        return

fish_manually()

https://farmrpg.com/#!/fishing.php?id=8
GOTO:  vast ocean -> fish
//div[@class="view view-main navbar-through"]//a[@href='fish.php']
[<selenium.webdriver.remote.webelement.WebElement (session="6aa26f90a6610b94ac20c57a65c4a987", element="85B6219981F6308D9A2566ECC172FCC0_element_921")>, <selenium.webdriver.remote.webelement.WebElement (session="6aa26f90a6610b94ac20c57a65c4a987", element="85B6219981F6308D9A2566ECC172FCC0_element_922")>, <selenium.webdriver.remote.webelement.WebElement (session="6aa26f90a6610b94ac20c57a65c4a987", element="85B6219981F6308D9A2566ECC172FCC0_element_923")>, <selenium.webdriver.remote.webelement.WebElement (session="6aa26f90a6610b94ac20c57a65c4a987", element="85B6219981F6308D9A2566ECC172FCC0_element_924")>, <selenium.webdriver.remote.webelement.WebElement (session="6aa26f90a6610b94ac20c57a65c4a987", element="85B6219981F6308D9A2566ECC172FCC0_element_925")>, <selenium.webdriver.remote.webelement.WebElement (session="6aa26f90a6610b94ac20c57a65c4a987", element="85B

In [ ]:
while True:
    harvest()
    plant_crop("carrot")
    ac = get_available_crops()
    if ac[ac["name"] == "Carrot Seeds"].iloc[0,1] < 100:
        pass
        # buy seeds
    time.sleep(121)


https://farmrpg.com/#!/xfarm.php?id=343807
https://farmrpg.com/#!/xfarm.php?id=343807
https://farmrpg.com/#!/xfarm.php?id=343807
Planted Carrot Seeds
https://farmrpg.com/#!/xfarm.php?id=343807


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=118.0.5993.118); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00D34DE3+43907]
	(No symbol) [0x00CC0741]
	(No symbol) [0x00BB33ED]
	(No symbol) [0x00BB6F6D]
	(No symbol) [0x00BB89D9]
	(No symbol) [0x00C15DEB]
	(No symbol) [0x00C02B5C]
	(No symbol) [0x00C155CA]
	(No symbol) [0x00C02956]
	(No symbol) [0x00BDE17E]
	(No symbol) [0x00BDF32D]
	GetHandleVerifier [0x00FE5AF9+2865305]
	GetHandleVerifier [0x0102E78B+3163435]
	GetHandleVerifier [0x01028441+3138017]
	GetHandleVerifier [0x00DBE0F0+605840]
	(No symbol) [0x00CCA64C]
	(No symbol) [0x00CC6638]
	(No symbol) [0x00CC675F]
	(No symbol) [0x00CB8DB7]
	BaseThreadInitThunk [0x761BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77D37C6E+286]
	RtlGetAppContainerNamedObjectPath [0x77D37C3E+238]
